<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Spark-Session" data-toc-modified-id="Spark-Session-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Spark Session</a></span></li><li><span><a href="#Spark-Dataframes" data-toc-modified-id="Spark-Dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Spark Dataframes</a></span><ul class="toc-item"><li><span><a href="#Dataframe-from-a-csv-file" data-toc-modified-id="Dataframe-from-a-csv-file-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Dataframe from a csv file</a></span></li></ul></li></ul></div>

# Imports

In [1]:
from pyspark.sql import SparkSession

# Spark Session

In [2]:
# instantiating a spark session

#conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])
spark = SparkSession.builder.getOrCreate()

# Spark Dataframes

## Dataframe from a csv file